In [ ]:
Task 1

In [146]:
import cv2
import numpy as np
import math

img = cv2.imread('noise.jpg',0)
cv2.imwrite('noisee.jpg', img)
print(img.shape)

a = 3    
b = 3    
ker = np.ones((a,b))
kernellen= a*b

def erosion(image , kernel ):
    erosimg = np.zeros(image.shape)
    rows = math.floor(kernel.shape[0]/2)
    cols = math.floor(kernel.shape[1]/2)
    for i in range(rows, image.shape[0] - rows):
        for j in range(cols, image.shape[1] - rows):
            temp = image[i - rows: i + rows + 1, j - cols: j + cols + 1]
            count = 0
            for k in range (kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    if(temp[k][l] == 255):
                        count += 1
            if (count == kernellen):
                erosimg[i][j] = 255
    return erosimg
                        
def dilation(image, kernel):
    erosimg = np.zeros(image.shape)
    rows = math.floor(kernel.shape[0]/2)
    cols = math.floor(kernel.shape[1]/2)
    for i in range(rows, image.shape[0] - rows):
        for j in range(cols, image.shape[1] - rows):
            temp = image[i - rows: i + rows + 1, j - cols: j + cols + 1]
            count = 0
            for k in range (kernel.shape[0]):
                for l in range(kernel.shape[1]):
                    if(temp[k][l] == 255):
                        count = 1
            if (count == 1):
                erosimg[i][j] = 255
    return erosimg


OTC = erosion(dilation(dilation(erosion(img,ker),ker),ker),ker)   
cv2.imwrite('Opening_Then_Closing.png',OTC)

CTO = dilation(erosion(erosion(dilation(img,ker),ker),ker),ker)   
cv2.imwrite('Closing_Then_Opening.png',CTO)

EOTC = erosion(OTC, ker)
ECTO = erosion(CTO, ker)
BoundaryOTC = OTC - EOTC
BoundaryCTO = CTO - ECTO

cv2.imwrite('Boundary_OTC.png', BoundaryOTC)
cv2.imwrite('Boundary_CTO.png', BoundaryCTO)


    

(310, 351)


True

Task 2

In [11]:
import cv2
import numpy as np
import math

kernel = np.array([[1, -1, 1], [-1, 4, -1], [-1, -1, -1]])
img_point = cv2.imread('turbine-blade.jpg', 0)


def convol(img, ker):
    convol_img = np.zeros((img.shape))
    img_sub = np.ndarray((ker.shape))
    m = img.shape[0]  # m-> no. of rows in Image
    n = img.shape[1]
    p = ker.shape[0]  # p -> no. of rows in Kernel
    q = ker.shape[1]
    rows = math.floor(p / 2)  # rows - > first row
    cols = math.floor(q / 2)
    for i in range(rows, m - rows):
        for j in range(cols, n - cols):
            img_sub = img[i - rows: i + rows + 1, j - cols: j + cols + 1]
            summ = 0.0
            for k in range(p):
                for l in range(q):
                    summ = summ + img_sub[k][l] * ker[k][l]
            if summ >329 :                                     #We do thresholding here, in convol itself
                #if we take threshold 312 we get 2 points
                print(summ, i, j)
                convol_img[i][j] = 255


    return (convol_img)

convoluted_img = convol(img_point, kernel)
cv2.imwrite('POINTS.jpg', convoluted_img)
print(img_point.shape)

font                   = cv2.FONT_HERSHEY_SIMPLEX
coordinates = (300,245)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

cv2.putText(convoluted_img,'Porosity', 
    coordinates, 
    font, 
    fontScale,
    fontColor,
    lineType)

cv2.imwrite("LabelledImage.jpg", convoluted_img)

338.0 250 445
(766, 571)


True

Task 2b

In [6]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches

img_seg = cv2.imread('segment.jpg', 0)
seg_threshold = np.zeros((img_seg.shape))

for i in range(img_seg.shape[0]):
    for j in range(img_seg.shape[1]):
        if img_seg[i][j]>204 :
            seg_threshold[i][j] = 255

cv2.imwrite('seg_threshold204.jpg', seg_threshold)  
i_coordinates = []                                      #all values of distinct i saved in list set wise
j_coordinates = []

for i in range(img_seg.shape[0]):
    for j in range(img_seg.shape[1]):
        if seg_threshold[i][j] == 255:
            j_coordinates.append(j)               #j_coordinates --> All values of j saved
j_coor = list(set(j_coordinates))                 #j_coor --> Distinct values of J coordinates saved
j_coorspaced = []                                 #j_coorspaced --> SAve the sets of J coordinates
j_coorspaced.append(j_coor[0])
for i in range(len(j_coor)-1):
    if j_coor[i+1]-j_coor[i] > 10 :
        j_coorspaced.append(j_coor[i])
        j_coorspaced.append(j_coor[i+1])
j_coorspaced.append(j_coor[len(j_coor)-1])

def icoor(seg_threshold, k, l):
    i_funcoor = []                       #i_funcoor-->to store all the i points and are returned later but 
    for i in range(img_seg.shape[0]):    #while returning only sets of distinct values are returned
        for j in range(k, l+1):
            if seg_threshold[i][j] == 255:
                i_funcoor.append(i)                                        
    return list(set(i_funcoor))

for i in range(math.floor(len(j_coorspaced)/2)):
    i_coordinates.append(icoor(seg_threshold, j_coorspaced[2*i], j_coorspaced[2*i+1]))   
                                                #sets of distinct values of i are appended in i_coordinates

i_coorspaced = []                                                #to store the top and bottom of segments
for i in range(len(i_coordinates)):  
    i_coorspaced.append(min(i_coordinates[i]))
    i_coorspaced.append(max(i_coordinates[i]))
    
cv2.rectangle(seg_threshold, (164, 127), (199, 162), (255,0,0), 2)
cv2.rectangle(seg_threshold, (255, 79), (300, 202), (255,0,0), 2)
cv2.rectangle(seg_threshold, (337, 27), (362, 284), (255,0,0), 2)
cv2.rectangle(seg_threshold, (390, 44), (420, 249), (255,0,0), 2)
cv2.imwrite("pahilabox1.jpg",seg_threshold)

True

In [7]:
i_coorspaced

[127, 162, 79, 202, 27, 284, 44, 249]

TASK 3

In [330]:
import cv2
import numpy as np
import math

img_hough = cv2.imread('hough.jpg', 0)
img_canny = cv2.Canny(img_hough, 100, 250)
cv2.imwrite("Img_canny.jpg",img_canny)

ker_hough = np.array([[-1, 2, -1], [-1, 2, -1], [-1, 2, -1]])

def convol(img, ker):
    convol_img = np.zeros((img.shape))
    img_sub = np.ndarray((ker.shape))
    m = img.shape[0]  # m-> no. of rows in Image
    n = img.shape[1]
    p = ker.shape[0]  # p -> no. of rows in Kernel
    q = ker.shape[1]
    rows = math.floor(p / 2)  # rows - > first row
    cols = math.floor(q / 2)
    for i in range(rows, m - rows):
        for j in range(cols, n - cols):
            img_sub = img[i - rows: i + rows + 1, j - cols: j + cols + 1]
            summ = 0.0
            for k in range(p):
                for l in range(q):
                    summ = summ + img_sub[k][l] * ker[k][l]       
            if summ > 1529:                                   #We do thresholding here itself
                convol_img[i][j] = 255
    return convol_img

convol_hough = convol(img_canny, ker_hough)
cv2.imwrite("Hough_ConvolThreshold.jpg",convol_hough)

rows = convol_hough.shape[0]
cols = convol_hough.shape[1]
diag_len = np.ceil(np.sqrt(rows * rows + cols * cols))

huff_matrix = np.zeros((int(diag_len), 181))

for i in range(convol_hough.shape[0]):
    for j in range(convol_hough.shape[1]):
        if convol_hough[i][j] == 255:
            for theta in range(0, 181):
                rho = int(np.round(i * np.cos(np.deg2rad(theta)) + j * np.sin(np.deg2rad(theta))))
                huff_matrix[rho, theta] += 1

max_rho = []
maxval = []
maxrho = []
max_rho = 0
maxtheta = []
max_theta = 0
maxvalue = 0
maxvalue2 = 0

for z in range(15):
    for i in range(huff_matrix.shape[0]):
        for j in range(huff_matrix.shape[1]):
            if huff_matrix[i, j] > maxvalue and z==0:
                maxvalue = huff_matrix[i, j]
                max_rho = i
                max_theta = j
            if huff_matrix[i, j] < maxvalue and huff_matrix[i, j] > maxvalue2 and z!=0:
                maxvalue2 = huff_matrix[i, j]
                max_rho = i
                max_theta = j
    if z==0:
        maxval.append(maxvalue)
    else:
        maxval.append(maxvalue2)
        maxvalue = maxvalue2
        maxvalue2 = 0
    maxrho.append(max_rho)
    maxtheta.append(max_theta)




In [332]:
maxval_unique = []
maxrho_unique = []
maxtheta_unique = []
indexes_unique = []
sortedrho = []

sortedrho = sorted(maxrho)
indexes_unique.append(sortedrho[0])
for i in range(len(sortedrho)-1):
    if(sortedrho[i+1] - sortedrho[i] > 30):
        indexes_unique.append(sortedrho[i+1])
for i in range(len(indexes_unique)):
    for j in range(len(maxrho)):
        if(maxrho[j] == indexes_unique[i]):
            maxval_unique.append(maxval[j])
            maxrho_unique.append(maxrho[j])
            maxtheta_unique.append(maxtheta[j])

[562, 182, 82, 470, 467, 86, 564, 376, 178, 181, 185, 278, 373, 565, 89]
[562, 182, 82, 470, 467, 86, 564, 376, 178, 181, 185, 278, 373, 565, 89]
[562, 182, 82, 470, 467, 86, 564, 376, 178, 181, 185, 278, 373, 565, 89]


In [344]:
vertical_lines = np.zeros(convol_hough.shape)
for z in range(6):
    for i in range(450):
        x = i
        y = math.ceil((-x/math.tan(math.radians(maxtheta_unique[z])))+maxrho_unique[z]/math.sin(math.radians(maxtheta_unique[z])))
        vertical_lines[x][y]=255
        convol_hough[x][y] = 255

    

In [345]:
cv2.imwrite("VerticalLineall.jpg",vertical_lines)
cv2.imwrite("HoughpeVerticalall.jpg", convol_hough)

True

Task 3b

In [12]:
import cv2
import numpy as np
import math

img_hough = cv2.imread('hough.jpg', 0)
img_canny = cv2.Canny(img_hough, 100, 250)
cv2.imwrite("Img_canny.jpg",img_canny)

ker_hough = np.array([[2, -1, -1], [-1, 2, -1], [-1, -1, 2]])

def convol(img, ker):
    convol_img = np.zeros((img.shape))
    img_sub = np.ndarray((ker.shape))
    m = img.shape[0]  # m-> no. of rows in Image
    n = img.shape[1]
    p = ker.shape[0]  # p -> no. of rows in Kernel
    q = ker.shape[1]
    rows = math.floor(p / 2)  # rows - > first row
    cols = math.floor(q / 2)
    for i in range(rows, m - rows):
        for j in range(cols, n - cols):
            img_sub = img[i - rows: i + rows + 1, j - cols: j + cols + 1]
            summ = 0.0
            for k in range(p):
                for l in range(q):
                    summ = summ + img_sub[k][l] * ker[k][l]       
            if summ > 900:                                   #We do thresholding here itself
                convol_img[i][j] = 255
    return convol_img

convol_hough = convol(img_canny, ker_hough)
cv2.imwrite("Hough_ConvolThreshold1020.jpg",convol_hough)


True

In [13]:
rows = convol_hough.shape[0]
cols = convol_hough.shape[1]
diag_len = np.ceil(np.sqrt(rows * rows + cols * cols))

huff_matrix = np.zeros((int(diag_len), 181))

for i in range(convol_hough.shape[0]):
    for j in range(convol_hough.shape[1]):
        if convol_hough[i][j] == 255:
            for theta in range(0, 181):
                rho = int(np.round(i * np.cos(np.deg2rad(theta)) + j * np.sin(np.deg2rad(theta))))
                huff_matrix[rho, theta] += 1

max_rho = []
maxval = []
maxrho = []
max_rho = 0
maxtheta = []
max_theta = 0
maxvalue = 0
maxvalue2 = 0

for z in range(15):
    for i in range(huff_matrix.shape[0]):
        for j in range(huff_matrix.shape[1]):
            if huff_matrix[i, j] > maxvalue and z==0:
                maxvalue = huff_matrix[i, j]
                max_rho = i
                max_theta = j
            if huff_matrix[i, j] < maxvalue and huff_matrix[i, j] > maxvalue2 and z!=0:
                maxvalue2 = huff_matrix[i, j]
                max_rho = i
                max_theta = j
    if z==0:
        maxval.append(maxvalue)
    else:
        maxval.append(maxvalue2)
        maxvalue = maxvalue2
        maxvalue2 = 0
    maxrho.append(max_rho)
    maxtheta.append(max_theta)


In [14]:
maxval_unique = []
maxrho_unique = []
maxtheta_unique = []
indexes_unique = []
sortedrho = []

sortedrho = sorted(maxrho)
indexes_unique.append(sortedrho[0])
for i in range(len(sortedrho)-1):
    if(sortedrho[i+1] - sortedrho[i] > 5):
        indexes_unique.append(sortedrho[i+1])
for i in range(len(indexes_unique)):
    for j in range(len(maxrho)):
        if(maxrho[j] == indexes_unique[i]):
            maxval_unique.append(maxval[j])
            maxrho_unique.append(maxrho[j])
            maxtheta_unique.append(maxtheta[j])

In [15]:
vertical_lines = np.zeros(convol_hough.shape)
for z in range(3):
    for i in range(450):
        x = i
        y = math.ceil((-x/math.tan(math.radians(maxtheta_unique[z])))+maxrho_unique[z]/math.sin(math.radians(maxtheta_unique[z])))
        vertical_lines[x][y]=255
        convol_hough[x][y] = 255
z = 3
for i in range(450):
        x = i
        y = math.ceil((-x/math.tan(math.radians(maxtheta_unique[z])))+maxrho_unique[z]/math.sin(math.radians(maxtheta_unique[z])))
        vertical_lines[x][y]=255
        convol_hough[x][y] = 255

temp1 = np.zeros(convol_hough.shape)
temp2 = np.zeros(convol_hough.shape)
tempfin = np.zeros(convol_hough.shape)
for i in range(350):
        x = i        
        y = math.ceil((-x/math.tan(math.radians(127)))+320/math.sin(math.radians(127)))
        temp2[x][y]=255
        y = math.ceil((-x/math.tan(math.radians(126)))+320/math.sin(math.radians(126)))
        temp1[x][y]=255
tempfin = temp2 - temp1
for i in range(tempfin.shape[0]):
    for j in range(tempfin.shape[1]):
        if tempfin[i][j] == 255 :
            vertical_lines[i][j]=255
            convol_hough[i][j] = 255
            

In [16]:
cv2.imwrite("VerticalLineall.jpg",vertical_lines)
cv2.imwrite("HoughpeVerticalall320-127.jpg", convol_hough)

True